In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
from shapely.geometry import Point

# exemplo de leitura do arquivo txt dentro da bilbioteca pandas

link https://automating-gis-processes.github.io/site/notebooks/L3/spatial_index.html

In [ ]:
raios19 = pd.read_csv('./raios_2019_filtrado.txt', 
                      sep=';', 
                      #skiprows=1, 
                      usecols=[0,2,3,4], 
                      names=['data_r', 'latr', 'lonr', 'pc'], 
                      low_memory=False,
#                       nrows=20000
                    )

In [ ]:
raios19.info()

In [ ]:
 raios19.tail(3) 

In [ ]:

# Prepara um novo atributo do tipo geometry utilizando a funcao Point do shapely para depois criar um geopandas
# esta variavel row eh soh pra ajudar a entender que ele vai pegar pra cada linha o atributo entre chaves

geometry = raios19.apply(lambda row: Point(row['lonr'], row['latr']), axis=1)

# cria uma variavel com referencia ao sistema de projecao cartografica
# crs = {'init': 'epsg:4326'}
crs=4326

# cria um geodataframe com o dataframe anteriro
# neste caso usando o mesmo nome mais pode ser outro qualquer
raio19_geo = gpd.GeoDataFrame(raios19, crs=crs, geometry=geometry)

# cria um indice espacial no geodataframe para agilizar os processamento espacial
 raio19_geo.sindex

In [ ]:
raio19_geo.has_sindex

In [ ]:
raio19_geo.info()

# Exporta o geodataframe para um shapefile

In [ ]:
# raio19_geo.to_file('raio_geo2.shp')
# raio19_geo.to_file('raio_geo.gpkg', driver='GPKG')

# exemplo de leitura de um shapefile dentro da biblioteca geopandas

In [2]:
# nome do arquivo shapefile
shape_name = 'RJ.shp'

In [3]:
shp_rj = gpd.read_file(shape_name)

In [4]:
# raio19_geo = gpd.read_file('raio_geo.shp', mask=shp_rj.geometry)
raio19_geo = gpd.read_file('raios_com_fogo_rj.gpkg', driver='GPKG', mask=shp_rj.geometry)

In [5]:
raio19_geo.sindex

In [6]:
raio19_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   data_r    10 non-null     object  
 1   latr      10 non-null     float64 
 2   lonr      10 non-null     float64 
 3   pc        10 non-null     int64   
 4   geometry  10 non-null     geometry
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 528.0+ bytes


# Teste de leitura de um shape de pontos filtrando na leitura

# plotagem de mapa
Apresenta os dados na forma de um mapa

In [ ]:
# mostrar os dois dados em uma mesma plotagem
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(24,12))

raio19_geo.plot(ax=ax)
shp_rj.plot(ax=ax, facecolor='None')


# Leitura dos focos

In [7]:
focos = pd.read_csv('Focos_npp_RJ_2019.csv')

In [8]:
# filtra apenas os campos de interesse do dataframe
focos = focos[['datahora', 'latitude', 'longitude']]

In [9]:
focos.rename(columns={'datahora': 'dataf', 'latitude':'latf', 'longitude':'lonf'}, inplace=True)

In [10]:
focos.head()

,dataf,latf,lonf
0,2019/07/22 16:06:00,-21.6802,-41.3524
1,2019/01/10 16:24:00,-22.9766,-43.6045
2,2019/01/04 16:36:00,-22.9682,-43.6756
3,2019/01/04 16:36:00,-22.9680,-43.6751
4,2019/01/04 16:36:00,-22.8889,-43.5091


In [11]:
# Cria um geodataframe dos focos
focos_geo = gpd.GeoDataFrame(focos, 
                             geometry=gpd.points_from_xy(focos.lonf, focos.latf), 
                             crs='EPSG:4326')

In [12]:
focos_geo.sindex

In [13]:
# cria um dados na forma de buffer redondo com raio de 500m representados em graus decimais (0.005)
# buff_focos = focos_geo.geometry.buffer(distance=0.005)

In [14]:
focos_geo['fbuffer_geom'] = focos_geo.buffer(distance=0.005)

/var/folders/m2/xln7ft1d69b42rc360vwsyhr0000gn/T/ipykernel_3598/2888589407.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  focos_geo['fbuffer_geom'] = focos_geo.buffer(distance=0.005)


In [15]:
buff_focos = focos_geo[['dataf', 'fbuffer_geom']].set_geometry('fbuffer_geom')

In [16]:
buff_focos.head()

,dataf,fbuffer_geom
0,2019/07/22 16:06:00,"POLYGON ((-41.34740 -21.68020, -41.34742 -21.6..."
1,2019/01/10 16:24:00,"POLYGON ((-43.59950 -22.97660, -43.59952 -22.9..."
2,2019/01/04 16:36:00,"POLYGON ((-43.67060 -22.96820, -43.67062 -22.9..."
3,2019/01/04 16:36:00,"POLYGON ((-43.67010 -22.96800, -43.67012 -22.9..."
4,2019/01/04 16:36:00,"POLYGON ((-43.50410 -22.88890, -43.50412 -22.8..."


In [ ]:
# exporta os dados dos buffers para um arquivo shapefile
# buff_focos.to_file('buffer_focos.shp')

# Plotagem de mapa
Apresenta os dados na forma de um mapa

In [ ]:
buff_focos.plot(figsize=(24,12));

In [ ]:
# mostrar os dois dados em uma mesma plotagem
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(24,12))

shp_rj.plot(ax=ax, facecolor='None')
raio19_geo.plot(ax=ax)
buff_focos.plot(ax=ax, facecolor='None');

# Filtragem dos raios que intersectam os buffers de focos

In [17]:
raios_focos = gpd.sjoin(raio19_geo, buff_focos,lsuffix='raio', rsuffix='focos')

# converte os campos de data para o tipo datetime

In [18]:
raios_focos['data_raio_dt'] = pd.to_datetime(raios_focos.data_r)

In [19]:
raios_focos['data_foco_dt'] = pd.to_datetime(raios_focos.dataf)

In [20]:
raios_focos.head()

,data_r,latr,lonr,pc,geometry,index_focos,dataf,data_raio_dt,data_foco_dt
0,2019-10-29 19:55:48,-22.5786,-44.6246,14,POINT (-44.62460 -22.57860),4486,2019/10/30 16:30:00,2019-10-29 19:55:48,2019-10-30 16:30:00
0,2019-10-29 19:55:48,-22.5786,-44.6246,14,POINT (-44.62460 -22.57860),4485,2019/10/30 16:30:00,2019-10-29 19:55:48,2019-10-30 16:30:00
1,2019-03-11 19:46:36,-22.4756,-42.7792,-14,POINT (-42.77920 -22.47560),642,2019/03/13 17:00:00,2019-03-11 19:46:36,2019-03-13 17:00:00
2,2019-03-11 19:42:49,-22.4808,-42.7772,-13,POINT (-42.77720 -22.48080),642,2019/03/13 17:00:00,2019-03-11 19:42:49,2019-03-13 17:00:00
3,2019-03-11 19:46:36,-22.4768,-42.7760,-43,POINT (-42.77600 -22.47680),642,2019/03/13 17:00:00,2019-03-11 19:46:36,2019-03-13 17:00:00


# Cria um intervalo de 48 horas a partir da data do foco

In [21]:
raios_focos['intervalo'] = raios_focos.data_foco_dt - pd.to_timedelta(48, unit='hours')

In [22]:
# if(to_datetime( "limite" )<=to_datetime( "DataR" ) and to_datetime( "DataR" )<=to_datetime( "DataF" ),5,7)
candidatos = raios_focos[(raios_focos.intervalo <= raios_focos.data_raio_dt) & 
            (raios_focos.data_raio_dt <= raios_focos.data_foco_dt)]

In [23]:
candidatos

,data_r,latr,lonr,pc,geometry,index_focos,dataf,data_raio_dt,data_foco_dt,intervalo
0,2019-10-29 19:55:48,-22.5786,-44.6246,14,POINT (-44.62460 -22.57860),4486,2019/10/30 16:30:00,2019-10-29 19:55:48,2019-10-30 16:30:00,2019-10-28 16:30:00
0,2019-10-29 19:55:48,-22.5786,-44.6246,14,POINT (-44.62460 -22.57860),4485,2019/10/30 16:30:00,2019-10-29 19:55:48,2019-10-30 16:30:00,2019-10-28 16:30:00
1,2019-03-11 19:46:36,-22.4756,-42.7792,-14,POINT (-42.77920 -22.47560),642,2019/03/13 17:00:00,2019-03-11 19:46:36,2019-03-13 17:00:00,2019-03-11 17:00:00
2,2019-03-11 19:42:49,-22.4808,-42.7772,-13,POINT (-42.77720 -22.48080),642,2019/03/13 17:00:00,2019-03-11 19:42:49,2019-03-13 17:00:00,2019-03-11 17:00:00
3,2019-03-11 19:46:36,-22.4768,-42.7760,-43,POINT (-42.77600 -22.47680),642,2019/03/13 17:00:00,2019-03-11 19:46:36,2019-03-13 17:00:00,2019-03-11 17:00:00
4,2019-03-11 19:46:36,-22.4801,-42.7745,-13,POINT (-42.77450 -22.48010),642,2019/03/13 17:00:00,2019-03-11 19:46:36,2019-03-13 17:00:00,2019-03-11 17:00:00
5,2019-03-05 19:48:32,-22.8398,-44.0852,-10,POINT (-44.08520 -22.83980),636,2019/03/07 17:12:00,2019-03-05 19:48:32,2019-03-07 17:12:00,2019-03-05 17:12:00
6,2019-01-15 21:41:18,-22.6260,-43.2218,-6,POINT (-43.22180 -22.62600),4807,2019/01/17 15:54:00,2019-01-15 21:41:18,2019-01-17 15:54:00,2019-01-15 15:54:00
7,2019-01-15 21:41:18,-22.6260,-43.2218,-7,POINT (-43.22180 -22.62600),4807,2019/01/17 15:54:00,2019-01-15 21:41:18,2019-01-17 15:54:00,2019-01-15 15:54:00
8,2019-01-16 21:11:21,-22.6223,-43.2279,-14,POINT (-43.22790 -22.62230),4807,2019/01/17 15:54:00,2019-01-16 21:11:21,2019-01-17 15:54:00,2019-01-15 15:54:00


In [24]:
# raios_unicos = 
candidatos.index.unique()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [ ]:
# exporta a listagem dos raios que intersectaram com focos
# não vai exportar raios repetidos

# raio19_geo.loc[candidatos.index.unique()].to_file('./raios_com_fogo_rj.gpkg', driver='GPKG')

# Verificação do raio mais próxio com a distância do centro do buffer do foco 

In [25]:
# Quantos raios estão contidos em cada buffer?
candidatos.index_focos.value_counts()

642     4
4807    3
4486    1
4485    1
636     1
4564    1
4547    1
Name: index_focos, dtype: int64

In [26]:
def resumo(df):
    rcount = pd.DataFrame(df.index_focos.value_counts())
    rcount.reset_index(inplace=True)
    rcount.rename(columns={"index": "idf", "index_focos": "cnt"}, inplace=True)
    return(rcount)

In [58]:
def getdistance(row):
    a = focos_geo.loc[focos_geo.index == 0].to_crs(epsg=32722)
    r = candidatos.loc[candidatos.index == 32735].to_crs(epsg=32722)
    d = r.distance(a,align=False)
    print(d)

In [69]:
a = focos_geo.loc[focos_geo.index == 642].to_crs(epsg=32722)
r = candidatos.loc[candidatos.index == 0].to_crs(epsg=32722)
d = r.distance(a,align=False)

/Users/fabianomorelli/opt/anaconda3/envs/geopy/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [32]:
type(focos_geo[focos_geo.index == 642])
# type(focos_geo.loc[642])

geopandas.geodataframe.GeoDataFrame

In [86]:
r.iloc[0]

data_r                               2019-10-29 19:55:48
latr                                            -22.5786
lonr                                            -44.6246
pc                                                    14
geometry        POINT (1156332.7201979 7489067.89068629)
index_focos                                         4486
dataf                                2019/10/30 16:30:00
data_raio_dt                         2019-10-29 19:55:48
data_foco_dt                         2019-10-30 16:30:00
intervalo                            2019-10-28 16:30:00
Name: 0, dtype: object

In [83]:
b = gpd.GeoDataFrame(r.iloc[0, ], geometry=r['geometry'], crs=4326)

/var/folders/m2/xln7ft1d69b42rc360vwsyhr0000gn/T/ipykernel_14669/4180017802.py:1: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  b = gpd.GeoDataFrame(r.iloc[0], geometry=r['geometry'], crs=4326)


ValueError: cannot reindex from a duplicate axis

In [81]:
a.distance(b, align=False)

AttributeError: No geometry data set yet (expected in column 'geometry'.)

In [51]:
rcount = resumo(candidatos)
rcount

,idf,cnt
0,642,4
1,4807,3
2,4486,1
3,4485,1
4,636,1
5,4564,1
6,4547,1


In [59]:
rcount.apply(getdistance, axis=1)

ValueError: Lengths of inputs do not match. Left: 0, Right: 1

In [ ]:
r = candidatos.loc[candidatos.index == 32735,'geometry'].to_crs(epsg=32722)

In [ ]:
#r.to_crs(epsg=32722).distance(a.to_crs(epsg=32722))
r.distance(a,align=False)